In [1]:
import numpy as np
import pandas as pd
from random import sample

import hyperopt
from hyperopt import hp, Trials
from hyperopt.pyll.base import scope

from sklearn.metrics import f1_score, log_loss, accuracy_score
from sklearn.utils.class_weight import compute_sample_weight

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

from coremdlr.models import NetworkModel
from coremdlr.config import defaults

from coremdlr.datasets import WellLoader, FaciesDataset
from coremdlr.datasets.utils import infer_test_wells

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
BCNN_SEARCH_SPACE = {
    'feature' : 'image',
    'network' : 'bilinear_cnn',
    'summary' : False,
    'sequence_size' : 12,
    'network_args' : {
        'backbone_cnn' : 'vgg16',
        'conv1x1' : scope.int(hp.quniform('conv1x1', 16, 32, 8)),
        'apply_rowwise' : True,
        'lstm_features' : None,
        'dropout_rate' : 0.5,
    },
    'optimizer_args' : {
        'optimizer' : 'Adam',
        'clipnorm' : 10.
    }
}

IMAGE_ARGS = {
    'image_width' : 200,
    'crop_method' : 'center',
}


fit_args = {
    'batch_size' : 4,
    'epochs' : 15,
    'class_weighted' : True
}

CHKPT_FNAME = 'temp_best_weights.h5'

In [3]:
def run_experiment(num_train, max_evals=15, search_space=BCNN_SEARCH_SPACE):
    
    assert num_train in [3, 6, 9], 'Must use one of predefined train/test splits'
    
    train_sets = getattr(defaults, f'TRAIN_SETS_{num_train}')
    
    #datasets = [FaciesDataset(s, infer_test_wells(s), features=['image'],
    #                          image_args=IMAGE_ARGS) for s in train_sets[:2]]
    
    #for dset in datasets:
    #    dset.load_or_generate_data()
    
    def train_model(model_config):
        
        acc_scores, f1_scores, log_losses = [], [], []
        
        for i, train_set in enumerate(train_sets):
            dset = FaciesDataset(train_set, infer_test_wells(train_set), features=['image'],
                                image_args=IMAGE_ARGS)
        
            dset.load_or_generate_data()
            
            model = NetworkModel(dset, model_args=model_config)
            
            chkpt_callback = ModelCheckpoint(CHKPT_FNAME, monitor='val_loss', verbose=1,
                                        save_best_only=True, 
                                        save_weights_only=True)
            earlystop_callback = EarlyStopping(monitor='val_loss', patience=3)
            fit_args['callbacks'] = [chkpt_callback, earlystop_callback]
            
            print(f'Training with config: {model_config}')
            
            model.fit(dset, **fit_args)
            
            model.network.load_weights(CHKPT_FNAME)
            
            y_proba = model.predict_proba(dset.X_test)
            y_pred = np.argmax(y_proba, -1)
            
            acc_scores.append(accuracy_score(dset.y_test, y_pred))
            
            f1_scores.append(f1_score(dset.y_test, y_pred, average='macro'))
            
            log_losses.append(log_loss(dset.y_test, y_proba, 
                                      sample_weight=compute_sample_weight('balanced', dset.y_test)))
            
        return {'loss' : np.mean(log_losses),
                'acc_scores' : acc_scores,
                'f1_scores' : f1_scores,
                'log_losses' : log_losses,
                'status' : hyperopt.STATUS_OK}
    
    
    trials = Trials()
    
    best_params = hyperopt.fmin(
        fn=train_model,
        space=BCNN_SEARCH_SPACE,
        algo=hyperopt.rand.suggest,
        max_evals=max_evals,
        trials=trials
    )
    
    return trials, best_params

In [ ]:
trails3, best_params3 = run_experiment(3, max_evals=3)

Loading Well:                                      
204-24a-7                                          
 from                                              
/home/ross/Dropbox/core_data/facies/train_data     
Feature shapes:                                    
[('depth', (19294,)), ('top', (19294,)), ('base', (19294,)), ('image', (19294, 32, 200, 3))]
Loading Well:                                      
204-24a-6                                          
 from                                              
/home/ross/Dropbox/core_data/facies/train_data     
Feature shapes:                                    
[('depth', (13006,)), ('top', (13006,)), ('base', (13006,)), ('image', (13006, 32, 200, 3))]
Loading Well:                                      
204-20-3                                           
 from                                              
/home/ross/Dropbox/core_data/facies/train_data     
Feature shapes:                                    
[('depth', (10906,)), ('top', (109

/home/ross/anaconda3/envs/core-dev/lib/python3.7/site-packages/sklearn/preprocessing/_function_transformer.py:97: FutureWarning: The default validate=True will be replaced by validate=False in 0.22.
  "validate=False in 0.22.", FutureWarning)



Loading Well:                                      
205-21b-3                                          
 from                                              
/home/ross/Dropbox/core_data/facies/train_data     
Feature shapes:                                    
[('depth', (3842,)), ('top', (3842,)), ('base', (3842,)), ('image', (3842, 32, 200, 3))]
Loading Well:                                      
204-20-1                                           
 from                                              
/home/ross/Dropbox/core_data/facies/train_data     
Feature shapes:                                    
[('depth', (11210,)), ('top', (11210,)), ('base', (11210,)), ('image', (11210, 32, 200, 3))]
Loading Well:                                      
204-19-7                                           
 from                                              
/home/ross/Dropbox/core_data/facies/train_data     
Feature shapes:                                    
[('depth', (9807,)), ('top', (9807,)),

/home/ross/anaconda3/envs/core-dev/lib/python3.7/site-packages/sklearn/preprocessing/_function_transformer.py:97: FutureWarning: The default validate=True will be replaced by validate=False in 0.22.
  "validate=False in 0.22.", FutureWarning)


W0723 16:08:07.696016 139966337451840 deprecation.py:506] From /home/ross/anaconda3/envs/core-dev/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor



Training with config: {'feature': 'image', 'network': 'bilinear_cnn', 'network_args': {'apply_rowwise': True, 'backbone_cnn': 'vgg16', 'conv1x1': 24, 'dropout_rate': 0.5, 'lstm_features': None}, 'optimizer_args': {'clipnorm': 10.0}, 'sequence_size': 12, 'summary': False}
Shapes of `(batch_X, batch_y)`: (4, 384, 200, 3), (4, 12, 5)
Epoch 1/15                                         
  0%|          | 0/3 [01:49<?, ?it/s, best loss: ?]

...

 - ETA: 5s - loss: 1.2019 - acc: 0.2879            
                                                   
1732/1799 [===========================>..]         
 - ETA: 5s - loss: 1.2019 - acc: 0.2879            
                                                   
1733/1799 [===========================>..]         
 - ETA: 5s - loss: 1.2019 - acc: 0.2878            
                                                   
1734/1799 [===========================>..]         
 - ETA: 4s - loss: 1.2019 - acc: 0.2879            
                                                   
1735/1799 [===========================>..]         
 - ETA: 4s - loss: 1.2019 - acc: 0.2879            
                                                   
1736/1799 [===========================>..]         
 - ETA: 4s - loss: 1.2017 - acc: 0.2880            
                                                   
1737/1799 [===========================>..]         
 - ETA: 4s - loss: 1.2017 - acc: 0.2879            
            

                                                   
1784/1799 [============================>.]         
 - ETA: 1s - loss: 1.2000 - acc: 0.2885            
                                                   
1785/1799 [============================>.]         
 - ETA: 1s - loss: 1.2000 - acc: 0.2886            
                                                   
1786/1799 [============================>.]         
 - ETA: 0s - loss: 1.1999 - acc: 0.2887            
                                                   
1787/1799 [============================>.]         
 - ETA: 0s - loss: 1.1997 - acc: 0.2888            
                                                   
1788/1799 [============================>.]         
 - ETA: 0s - loss: 1.1997 - acc: 0.2887            
                                                   
1789/1799 [============================>.]         
 - ETA: 0s - loss: 1.1999 - acc: 0.2887            
                                                   
1790/1799 [=

Exception in thread Thread-5:
Traceback (most recent call last):
  File "/home/ross/anaconda3/envs/core-dev/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/home/ross/anaconda3/envs/core-dev/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ross/anaconda3/envs/core-dev/lib/python3.7/site-packages/tensorflow/python/keras/utils/data_utils.py", line 764, in _run
    self.sequence.on_epoch_end()
  File "/home/ross/code/python/coremdlr/coremdlr/datasets/generators.py", line 105, in on_epoch_end
    self.X = self.X[p]
MemoryError



                                                   
Epoch 00002: val_loss improved from 1.92641 to 1.22523, saving model to temp_best_weights.h5
                                                   
1799/1799 [==============================]         
 - 213s 118ms/step - loss: 1.2003 - acc: 0.2884 - val_loss: 1.2252 - val_acc: 0.3065

Epoch 3/15                                         
  0%|          | 0/3 [09:01<?, ?it/s, best loss: ?]

Exception in thread Thread-19:
Traceback (most recent call last):
  File "/home/ross/anaconda3/envs/core-dev/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/home/ross/anaconda3/envs/core-dev/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ross/anaconda3/envs/core-dev/lib/python3.7/site-packages/tensorflow/python/keras/utils/data_utils.py", line 764, in _run
    self.sequence.on_epoch_end()
  File "/home/ross/code/python/coremdlr/coremdlr/datasets/generators.py", line 105, in on_epoch_end
    self.X = self.X[p]
MemoryError



In [ ]:
trails3.results

In [ ]:
best_params3

In [ ]:
trails3.best_trial

In [ ]:
def pred_results_df(num_train, model_config):
    
    assert num_train in [3, 6, 9], 'Must use one of predefined train/test splits'
    
    train_sets = getattr(defaults, f'TRAIN_SETS_{num_train}')
    
    pred_dfs = []
    
    for i, train_set in enumerate(train_sets):
        dset = FaciesDataset(train_set, infer_test_wells(train_set), features=['pseudoGR'], 
                             pseudoGR_args={'features' : ['mean', 'var'], 'per_channel' : True})
        
        dset.load_or_generate_data()
        
        model = NetworkModel(dset, model_args=model_config)
        
        chkpt_callback = ModelCheckpoint(CHKPT_FNAME, monitor='val_loss', verbose=1,
                                        save_best_only=True, 
                                        save_weights_only=True)
        
        model.fit(dset, callbacks=[chkpt_callback], verbose=False, **fit_args)
        
        model.network.load_weights(CHKPT_FNAME)
        
        #print('predict_proba: ', model.predict_proba(dset.X_test))
        
        for test_well_name in dset.test_well_names:
            df = model.preds_dataframe(test_well_name)
            df['well'] = test_well_name
            df['set_number'] = i
            pred_dfs.append(df)
        
    return pd.concat(pred_dfs)

In [ ]:
params = {
    'feature' : 'pseudoGR',
    'network' : 'wavenet',
    'summary' : False,
    'sequence_size' : 128,
    'network_args' : {
        'num_blocks' : 8,
        'block_filters' : 208,
        'residual_filters' : 40,
        'output_resolution' : 32,
        'dropout_rate' : 0.5,
    },
    'optimizer_args' : {
        'optimizer' : 'Adam'
    }
}

preds_df_3 = pred_results_df(3, params)
preds_df_3.to_csv('pgr_wavenet_results_train_sets3.csv')

In [ ]:
accuracy_score(preds_df_3.y_pred.values, preds_df_3.y_true.values)